<a href="https://colab.research.google.com/github/Alexhl09/RedNeuronalMCS/blob/main/RedEquipoMoradot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Favor de incluir los archivos 'dataSpotify.csv' y 'testData.csv' estan en el drive en la carpeta Tensorflow")

Favor de incluir los archivos 'dataSpotify.csv' y 'testData.csv' estan en el drive en la carpeta Tensorflow


In [ ]:

import TensorFlow
import PythonKit
import Foundation

%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/apple/swift-protobuf.git", from: "1.6.0")' SwiftProtobuf
import SwiftProtobuf



Installing packages:
	.package(url: "https://github.com/apple/swift-protobuf.git", from: "1.6.0")
		SwiftProtobuf
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpkfb_2cap/swift-install
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.12.0
[1/2] Compiling SwiftProtobuf AnyMessageStorage.swift
[2/3] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[3/3] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [ ]:
let hiddenSize: Int = 16
struct SongModel: Layer {
    var layer1 = Dense<Float64>(inputSize: 11, outputSize: hiddenSize, activation: relu)
    var layer2 = Dense<Float64>(inputSize: hiddenSize, outputSize: hiddenSize, activation: relu)
    var layer3 = Dense<Float64>(inputSize: hiddenSize, outputSize: 2)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float64>) -> Tensor<Float64> {
        return input.sequenced(through: layer1, layer2,layer3)
    }
}



var model = SongModel()


In [ ]:

let plt = Python.import("matplotlib.pyplot")
let trainDataFilename = "dataSpotify.csv"

let batchSize = 32

struct SongsBatch {

    let features: Tensor<Float64>

    let labels: Tensor<Int32>
}

extension SongsBatch: Collatable {
    public init<BatchSamples: Collection>(collating samples: BatchSamples)
        where BatchSamples.Element == Self {
        features = Tensor<Float64>(stacking: samples.map{$0.features})
        labels = Tensor<Int32>(stacking: samples.map{$0.labels})
    }
}


In [ ]:
func loadSongsDatasetFromCSV(
    contentsOf: String, hasHeader: Bool, featureColumns: [Int], labelColumns: [Int]) -> [SongsBatch] {

    let pd = Python.import("pandas")
    let np = Python.import("numpy")
    
    let pandasFea = pd.read_csv(contentsOf,
                                delimiter: ",",
                                usecols: featureColumns,
                                skiprows: hasHeader ? 1 : 0,
                                dtype: "float64")
   
    let featuresNp =  pandasFea.to_numpy()
    guard let featuresTensor = Tensor<Float64>(numpy: np.squeeze(featuresNp)) else {
            // This should never happen, because we construct featuresNp in such a
            // way that it should be convertible to tensor.
            fatalError("pd.read_csv result can't be converted to Tensor")
        }

    let pdLabel = pd.read_csv(
           contentsOf,
            delimiter: ",",
            skiprows: hasHeader ? 1 : 0,
            usecols: labelColumns,
            dtype: "int32")
       
    
    let labelNp =  pdLabel.to_numpy()
    
    guard let labelsTensor = Tensor<Int32>(numpy: np.squeeze(labelNp)) else {
            // This should never happen, because we construct featuresNp in such a
            // way that it should be convertible to tensor.
            fatalError("pd.read_csv result can't be converted to Tensor")
        }
    
    return zip(featuresTensor.unstacked(), labelsTensor.unstacked()).map{SongsBatch(features: $0.0, labels: $0.1)}



    }

In [ ]:

Context.local.learningPhase = .training

let trainingDataset: [SongsBatch] = loadSongsDatasetFromCSV(contentsOf: trainDataFilename,
                                                  hasHeader: true,
                                                  featureColumns: [0,2,4,7,9,10,13,15,16,17,18],
                                                  labelColumns: [5])


let trainingEpochs: TrainingEpochs = TrainingEpochs(samples: trainingDataset, batchSize: batchSize)

let firstTrainEpoch = trainingEpochs.next()!
let firstTrainBatch = firstTrainEpoch.first!.collated
let firstTrainFeatures = firstTrainBatch.features
let firstTrainLabels = firstTrainBatch.labels

print("First batch of features: \(firstTrainFeatures)")
print("firstTrainFeatures.shape: \(firstTrainFeatures.shape)")
print("First batch of labels: \(firstTrainLabels)")
print("firstTrainLabels.shape: \(firstTrainLabels.shape)")

let firstTrainFeaturesTransposed = firstTrainFeatures.transposed()
let loudness = firstTrainFeaturesTransposed[1].scalars
let danceability = firstTrainFeaturesTransposed[6].scalars

//plt.scatter(loudness, danceability, c: firstTrainLabels.array.scalars)
//plt.xlabel("Acurstica")
//plt.ylabel("Bailabilidad")
//plt.show()

First batch of features: [[                 0.946,    0.48200000000000004,                  0.255,
                9.67e-05,                  0.107,                 -9.793,
                    56.0,                  0.031,                 95.205,
                   0.301,                 1985.0],
 [                 0.862,                  0.508,    0.43799999999999994,
                0.000122,                  0.299,                -10.075,
                     0.0,                 0.0325,      75.94800000000001,
                   0.435,                 1930.0],
 [0.00045599999999999997,                  0.519,                  0.892,
                0.000525,     0.5720000000000001,                 -5.285,
                    45.0,                  0.113,     174.02700000000002,
      0.5589999999999999,                 2011.0],
 [                0.0839,     0.5870000000000001,                  0.679,
  0.00031600000000000004,                  0.413,                  -6.26,
        

In [ ]:


let firstTrainPredictions = model(firstTrainFeatures)
print(firstTrainPredictions[0..<5])
print(softmax(firstTrainPredictions[0..<5]))

print("Prediction: \(firstTrainPredictions.argmax(squeezingAxis: 1))")
print("    Labels: \(firstTrainLabels)")

let untrainedLogits = model(firstTrainFeatures)
print(untrainedLogits)
print(firstTrainLabels)
let untrainedLoss = softmaxCrossEntropy(logits: untrainedLogits, labels: firstTrainLabels)
print("Loss test: \(untrainedLoss)")
let optimizer = SGD(for: model, learningRate: 0.01)

let (loss, grads) = valueWithGradient(at: model) { model -> Tensor<Float64> in
    let logits = model(firstTrainFeatures)
    return softmaxCrossEntropy(logits: logits, labels: firstTrainLabels)
}
print("Current loss: \(loss)")

optimizer.update(&model, along: grads)

let logitsAfterOneStep = model(firstTrainFeatures)
let lossAfterOneStep = softmaxCrossEntropy(logits: logitsAfterOneStep, labels: firstTrainLabels)
print("Next loss: \(lossAfterOneStep)")


[[-261.56785505739924, -236.45017576120824],
 [-251.54328533759846, -224.63674141141996],
 [-300.72499432514974, -250.80914094370786],
 [ -303.2118994337786, -249.30214108017552],
 [ -294.5755891154463, -246.11747250190737]]
[[   1.2346119704353e-11,     0.9999999999876539],
 [2.0636519195076535e-12,     0.9999999999979363],
 [ 2.098071657266141e-22,                    1.0],
 [ 3.866245489004582e-24,                    1.0],
 [ 9.013788246154787e-22,                    1.0]]
Prediction: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    Labels: [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[[-261.56785505739924, -236.45017576120824],
 [-251.54328533759846, -224.63674141141996],
 [-300.72499432514974, -250.80914094370786],
 [ -303.2118994337786, -249.30214108017552],
 [ -294.5755891154463, -246.11747250190737],
 [-271.88223923055756,   -237.842013357645],
 [-274.84784819950727,  -237.916

In [ ]:


let epochCount = 50
var trainAccuracyResults: [Float] = []
var trainLossResults: [Float] = []

func accuracy(predictions: Tensor<Int32>, truths: Tensor<Int32>) -> Float {
    return Tensor<Float>(predictions .== truths).mean().scalarized()
}

print("Starting training...")

for (epochIndex, epoch) in trainingEpochs.prefix(epochCount).enumerated() {
    var epochLoss: Float = 0
    var epochAccuracy: Float = 0
    var batchCount: Int = 0
    for batchSamples in epoch {
        let batch = batchSamples.collated
        let (loss, grad) = valueWithGradient(at: model) { (model: SongModel) -> Tensor<Float64> in
            let logits = model(batch.features)
            return softmaxCrossEntropy(logits: logits, labels: batch.labels)
        }
        optimizer.update(&model, along: grad)

        let logits = model(batch.features)
        epochAccuracy += accuracy(predictions: logits.argmax(squeezingAxis: 1), truths: batch.labels)
        epochLoss += Float(loss.scalarized())
        batchCount += 1
    }
    epochAccuracy /= Float(batchCount)
    epochLoss /= Float(batchCount)
    trainAccuracyResults.append(epochAccuracy)
    trainLossResults.append(epochLoss)
    if epochIndex % 2 == 0 {
        print("Epoch \(epochIndex): Loss: \(epochLoss), Accuracy: \(epochAccuracy)")
    }
}

plt.figure(figsize: [12, 8])

let accuracyAxes = plt.subplot(2, 1, 1)
accuracyAxes.set_ylabel("Accuracy")
accuracyAxes.plot(trainAccuracyResults)

let lossAxes = plt.subplot(2, 1, 2)
lossAxes.set_ylabel("Loss")
lossAxes.set_xlabel("Epoch")
lossAxes.plot(trainLossResults)

plt.show()


Starting training...
Epoch 0: Loss: 10.492303, Accuracy: 0.8922417
Epoch 2: Loss: 0.34181798, Accuracy: 0.8922417
Epoch 4: Loss: 0.3417979, Accuracy: 0.8922507
Epoch 6: Loss: 0.34183154, Accuracy: 0.8922417
Epoch 8: Loss: 0.341859, Accuracy: 0.89222366
Epoch 10: Loss: 0.34188533, Accuracy: 0.8922146
Epoch 12: Loss: 0.34184325, Accuracy: 0.89222366
Epoch 14: Loss: 0.34181875, Accuracy: 0.8922417
Epoch 16: Loss: 0.34184593, Accuracy: 0.89223266
Epoch 18: Loss: 0.34186476, Accuracy: 0.89222366
Epoch 20: Loss: 0.3418479, Accuracy: 0.89223266
Epoch 22: Loss: 0.3418428, Accuracy: 0.89223266
Epoch 24: Loss: 0.34187353, Accuracy: 0.8922146
Epoch 26: Loss: 0.34182477, Accuracy: 0.8922417
Epoch 28: Loss: 0.3418403, Accuracy: 0.89223266
Epoch 30: Loss: 0.34186092, Accuracy: 0.89222366
Epoch 32: Loss: 0.34181103, Accuracy: 0.8922417
Epoch 34: Loss: 0.34180918, Accuracy: 0.8922507
Epoch 36: Loss: 0.3418362, Accuracy: 0.89223266
Epoch 38: Loss: 0.34178236, Accuracy: 0.8922598
Epoch 40: Loss: 0.34184

None


In [ ]:


print("Evaluating model...")

Context.local.learningPhase = .inference

let testDataFilename = "testData.csv"

let testDataset = loadSongsDatasetFromCSV(
    contentsOf: testDataFilename, hasHeader: true,
    featureColumns: [0,2,4,7,9,10,13,15,16,17,18], labelColumns: [5]).inBatches(of: batchSize)
// NOTE: Only a single batch will run in the loop since the batchSize we're using is larger than the test set size
for batchSamples in testDataset {
    let batch = batchSamples.collated
    let logits = model(batch.features)
    let predictions = logits.argmax(squeezingAxis: 1)
    print("Test batch accuracy: \(accuracy(predictions: predictions, truths: batch.labels))")
}



let firstTestBatch = testDataset.first!.collated
let firstTestBatchLogits = model(firstTestBatch.features)
let firstTestBatchPredictions = firstTestBatchLogits.argmax(squeezingAxis: 1)

print(firstTestBatchPredictions)
print(firstTestBatch.labels)

Evaluating model...
Test batch accuracy: 1.0
Test batch accuracy: 1.0
Test batch accuracy: 1.0
Test batch accuracy: 0.96875
Test batch accuracy: 1.0
Test batch accuracy: 0.9375
Test batch accuracy: 1.0
Test batch accuracy: 0.9375
Test batch accuracy: 1.0
Test batch accuracy: 0.96875
Test batch accuracy: 1.0
Test batch accuracy: 1.0
Test batch accuracy: 0.96875
Test batch accuracy: 0.96875
Test batch accuracy: 1.0
Test batch accuracy: 1.0
Test batch accuracy: 0.96875
Test batch accuracy: 0.96875
Test batch accuracy: 0.96875
Test batch accuracy: 0.90625
Test batch accuracy: 0.96875
Test batch accuracy: 0.96875
Test batch accuracy: 1.0
Test batch accuracy: 0.96875
Test batch accuracy: 0.96875
Test batch accuracy: 0.96875
Test batch accuracy: 0.96875
Test batch accuracy: 0.9375
Test batch accuracy: 1.0
Test batch accuracy: 1.0
Test batch accuracy: 1.0
Test batch accuracy: 1.0
Test batch accuracy: 0.96875
Test batch accuracy: 1.0
Test batch accuracy: 0.96875
Test batch accuracy: 0.90625
Tes

In [ ]:
print("Prediciendo")


//
//let unlabeledDataset: Tensor<Float64> =
//    [[0.36,  0.99, 0.618, 0.087,-8.07,  37.9, 0.0815, 120.78, 0.58],
//     [0.36,  0.568, 0.618, 0.087,-8.07,  37.9, 0.0815, 120.78, 0.58],
//     [0.36,  0.1, 0.618, 0.087,-8.07,  37.9, 0.0815, 120.78, 0.58]]
//
//let unlabeledDatasetPredictions = model(unlabeledDataset)
//
//for i in 0..<unlabeledDatasetPredictions.shape[0] {
//    let logits = unlabeledDatasetPredictions[i]
//    let classIdx = logits.argmax().scalar!
//    print("Example \(i) prediction: \(classNames[Int(classIdx)]) (\(softmax(logits)))")
//}
//


Prediciendo


In [ ]:

%include "ArrayFeatureExtractor.pb.swift"

In [ ]:
%include "BayesianProbitRegressor.pb.swift"

In [ ]:
%include "DataStructures.pb.swift"

In [ ]:
%include "CategoricalMapping.pb.swift"

In [ ]:
%include "CustomModel.pb.swift"

In [ ]:
%include "DictVectorizer.pb.swift"

In [ ]:
%include "FeatureTypes.pb.swift"

In [ ]:
%include "FeatureVectorizer.pb.swift"

In [ ]:
%include "GLMClassifier.pb.swift"

In [ ]:
%include "GLMRegressor.pb.swift"

In [ ]:
%include "Gazetteer.pb.swift"

In [ ]:
%include "Identity.pb.swift"

In [ ]:
%include "Imputer.pb.swift"

In [ ]:
%include "Scaler.pb.swift"

In [ ]:
%include "ItemSimilarityRecommender.pb.swift"

In [ ]:
%include "Parameters.pb.swift"

In [ ]:
%include "Normalizer.pb.swift"

In [ ]:
%include "LinkedModel.pb.swift"

In [ ]:
%include "NearestNeighbors.pb.swift"

In [ ]:
%include "NonMaximumSuppression.pb.swift"

In [ ]:
%include "OneHotEncoder.pb.swift"

In [ ]:
%include "SVM.pb.swift"

In [ ]:
%include "SoundAnalysisPreprocessing.pb.swift"

In [ ]:
%include "TextClassifier.pb.swift"

In [ ]:
%include "TreeEnsemble.pb.swift"

In [ ]:
%include "VisionFeaturePrint.pb.swift"

In [ ]:
%include "WordEmbedding.pb.swift"

In [ ]:
%include "WordTagger.pb.swift"

In [ ]:
%include "NeuralNetwork.pb.swift"

In [ ]:
%include "Model.pb.swift"

In [82]:
let pesosLayer11 : [Float] = [Float(Double(model.layer1.weight[0][0])!), Float(Double(model.layer1.weight[0][1])!),  Float(Double(model.layer1.weight[0][2])!), Float(Double(model.layer1.weight[0][3])!), Float(Double(model.layer1.weight[0][4])!),Float(Double(model.layer1.weight[0][5])!), Float(Double(model.layer1.weight[0][6])!),Float(Double(model.layer1.weight[0][7])!),Float(Double(model.layer1.weight[0][8])!),Float(Double(model.layer1.weight[0][9])!),Float(Double(model.layer1.weight[0][10])!),Float(Double(model.layer1.weight[0][11])!),Float(Double(model.layer1.weight[0][12])!), Float(Double(model.layer1.weight[0][13])!), Float(Double(model.layer1.weight[0][14])!), Float(Double(model.layer1.weight[0][15])!)]
let pesosLayer12 : [Float] = [Float(Double(model.layer1.weight[1][0])!), Float(Double(model.layer1.weight[1][1])!),  Float(Double(model.layer1.weight[1][2])!), Float(Double(model.layer1.weight[1][3])!), Float(Double(model.layer1.weight[1][4])!),Float(Double(model.layer1.weight[1][5])!), Float(Double(model.layer1.weight[1][6])!),Float(Double(model.layer1.weight[1][7])!),Float(Double(model.layer1.weight[1][8])!),Float(Double(model.layer1.weight[1][9])!),Float(Double(model.layer1.weight[1][10])!),Float(Double(model.layer1.weight[1][11])!),Float(Double(model.layer1.weight[1][12])!), Float(Double(model.layer1.weight[1][13])!), Float(Double(model.layer1.weight[1][14])!), Float(Double(model.layer1.weight[1][15])!)]
let pesosLayer13 : [Float] = [Float(Double(model.layer1.weight[2][0])!), Float(Double(model.layer1.weight[2][1])!),  Float(Double(model.layer1.weight[2][2])!), Float(Double(model.layer1.weight[2][3])!), Float(Double(model.layer1.weight[2][4])!),Float(Double(model.layer1.weight[2][5])!), Float(Double(model.layer1.weight[2][6])!),Float(Double(model.layer1.weight[2][7])!),Float(Double(model.layer1.weight[2][8])!),Float(Double(model.layer1.weight[2][9])!),Float(Double(model.layer1.weight[2][10])!),Float(Double(model.layer1.weight[2][11])!),Float(Double(model.layer1.weight[2][12])!), Float(Double(model.layer1.weight[2][13])!), Float(Double(model.layer1.weight[2][14])!), Float(Double(model.layer1.weight[2][15])!)]
let pesosLayer14 : [Float] = [Float(Double(model.layer1.weight[3][0])!), Float(Double(model.layer1.weight[3][1])!),  Float(Double(model.layer1.weight[3][2])!), Float(Double(model.layer1.weight[3][3])!), Float(Double(model.layer1.weight[3][4])!),Float(Double(model.layer1.weight[3][5])!), Float(Double(model.layer1.weight[3][6])!),Float(Double(model.layer1.weight[3][7])!),Float(Double(model.layer1.weight[3][8])!),Float(Double(model.layer1.weight[3][9])!),Float(Double(model.layer1.weight[3][10])!),Float(Double(model.layer1.weight[3][11])!),Float(Double(model.layer1.weight[3][12])!), Float(Double(model.layer1.weight[3][13])!), Float(Double(model.layer1.weight[3][14])!), Float(Double(model.layer1.weight[3][15])!)]
let pesosLayer15 : [Float] = [Float(Double(model.layer1.weight[4][0])!), Float(Double(model.layer1.weight[4][1])!),  Float(Double(model.layer1.weight[4][2])!), Float(Double(model.layer1.weight[4][3])!), Float(Double(model.layer1.weight[4][4])!),Float(Double(model.layer1.weight[4][5])!), Float(Double(model.layer1.weight[4][6])!),Float(Double(model.layer1.weight[4][7])!),Float(Double(model.layer1.weight[4][8])!),Float(Double(model.layer1.weight[4][9])!),Float(Double(model.layer1.weight[4][10])!),Float(Double(model.layer1.weight[4][11])!),Float(Double(model.layer1.weight[4][12])!), Float(Double(model.layer1.weight[4][13])!), Float(Double(model.layer1.weight[4][14])!), Float(Double(model.layer1.weight[4][15])!)]
let pesosLayer16 : [Float] = [Float(Double(model.layer1.weight[5][0])!), Float(Double(model.layer1.weight[5][1])!),  Float(Double(model.layer1.weight[5][2])!), Float(Double(model.layer1.weight[5][3])!), Float(Double(model.layer1.weight[5][4])!),Float(Double(model.layer1.weight[5][5])!), Float(Double(model.layer1.weight[5][6])!),Float(Double(model.layer1.weight[5][7])!),Float(Double(model.layer1.weight[5][8])!),Float(Double(model.layer1.weight[5][9])!),Float(Double(model.layer1.weight[5][10])!),Float(Double(model.layer1.weight[5][11])!),Float(Double(model.layer1.weight[5][12])!), Float(Double(model.layer1.weight[5][13])!), Float(Double(model.layer1.weight[5][14])!), Float(Double(model.layer1.weight[5][15])!)]
let pesosLayer17 : [Float] = [Float(Double(model.layer1.weight[6][0])!), Float(Double(model.layer1.weight[6][1])!),  Float(Double(model.layer1.weight[6][2])!), Float(Double(model.layer1.weight[6][3])!), Float(Double(model.layer1.weight[6][4])!),Float(Double(model.layer1.weight[6][5])!), Float(Double(model.layer1.weight[6][6])!),Float(Double(model.layer1.weight[6][7])!),Float(Double(model.layer1.weight[6][8])!),Float(Double(model.layer1.weight[6][9])!),Float(Double(model.layer1.weight[6][10])!),Float(Double(model.layer1.weight[6][11])!),Float(Double(model.layer1.weight[6][12])!), Float(Double(model.layer1.weight[6][13])!), Float(Double(model.layer1.weight[6][14])!), Float(Double(model.layer1.weight[6][15])!)]
let pesosLayer18 : [Float] = [Float(Double(model.layer1.weight[7][0])!), Float(Double(model.layer1.weight[7][1])!),  Float(Double(model.layer1.weight[7][2])!), Float(Double(model.layer1.weight[7][3])!), Float(Double(model.layer1.weight[7][4])!),Float(Double(model.layer1.weight[7][5])!), Float(Double(model.layer1.weight[7][6])!),Float(Double(model.layer1.weight[7][7])!),Float(Double(model.layer1.weight[7][8])!),Float(Double(model.layer1.weight[7][9])!),Float(Double(model.layer1.weight[7][10])!),Float(Double(model.layer1.weight[7][11])!),Float(Double(model.layer1.weight[7][12])!), Float(Double(model.layer1.weight[7][13])!), Float(Double(model.layer1.weight[7][14])!), Float(Double(model.layer1.weight[7][15])!)]
let pesosLayer19 : [Float] = [Float(Double(model.layer1.weight[8][0])!), Float(Double(model.layer1.weight[8][1])!),  Float(Double(model.layer1.weight[8][2])!), Float(Double(model.layer1.weight[8][3])!), Float(Double(model.layer1.weight[8][4])!),Float(Double(model.layer1.weight[8][5])!), Float(Double(model.layer1.weight[8][6])!),Float(Double(model.layer1.weight[8][7])!),Float(Double(model.layer1.weight[8][8])!),Float(Double(model.layer1.weight[8][9])!),Float(Double(model.layer1.weight[8][10])!),Float(Double(model.layer1.weight[8][11])!),Float(Double(model.layer1.weight[8][12])!), Float(Double(model.layer1.weight[8][13])!), Float(Double(model.layer1.weight[8][14])!), Float(Double(model.layer1.weight[8][15])!)]
let pesosLayer110 : [Float] = [Float(Double(model.layer1.weight[9][0])!), Float(Double(model.layer1.weight[9][1])!),  Float(Double(model.layer1.weight[9][2])!), Float(Double(model.layer1.weight[9][3])!), Float(Double(model.layer1.weight[9][4])!),Float(Double(model.layer1.weight[9][5])!), Float(Double(model.layer1.weight[9][6])!),Float(Double(model.layer1.weight[9][7])!),Float(Double(model.layer1.weight[9][8])!),Float(Double(model.layer1.weight[9][9])!),Float(Double(model.layer1.weight[9][10])!),Float(Double(model.layer1.weight[9][11])!),Float(Double(model.layer1.weight[9][12])!), Float(Double(model.layer1.weight[9][13])!), Float(Double(model.layer1.weight[9][14])!), Float(Double(model.layer1.weight[9][15])!)]
let pesosLayer111 : [Float] = [Float(Double(model.layer1.weight[10][0])!), Float(Double(model.layer1.weight[10][1])!),  Float(Double(model.layer1.weight[10][2])!), Float(Double(model.layer1.weight[10][3])!), Float(Double(model.layer1.weight[10][4])!),Float(Double(model.layer1.weight[10][5])!), Float(Double(model.layer1.weight[10][6])!),Float(Double(model.layer1.weight[10][7])!),Float(Double(model.layer1.weight[10][8])!),Float(Double(model.layer1.weight[10][9])!),Float(Double(model.layer1.weight[10][10])!),Float(Double(model.layer1.weight[10][11])!),Float(Double(model.layer1.weight[10][12])!), Float(Double(model.layer1.weight[10][13])!), Float(Double(model.layer1.weight[10][14])!), Float(Double(model.layer1.weight[10][15])!)]


let pesosLayer21 : [Float] = [Float(Double(model.layer2.weight[0][0])!), Float(Double(model.layer2.weight[0][1])!),  Float(Double(model.layer2.weight[0][2])!), Float(Double(model.layer2.weight[0][3])!), Float(Double(model.layer2.weight[0][4])!),Float(Double(model.layer2.weight[0][5])!), Float(Double(model.layer2.weight[0][6])!),Float(Double(model.layer2.weight[0][7])!),Float(Double(model.layer2.weight[0][8])!),Float(Double(model.layer2.weight[0][9])!),Float(Double(model.layer2.weight[0][10])!),Float(Double(model.layer2.weight[0][11])!),Float(Double(model.layer2.weight[0][12])!), Float(Double(model.layer2.weight[0][13])!), Float(Double(model.layer2.weight[0][14])!), Float(Double(model.layer2.weight[0][15])!)]
let pesosLayer22 : [Float] = [Float(Double(model.layer2.weight[1][0])!), Float(Double(model.layer2.weight[1][1])!),  Float(Double(model.layer2.weight[1][2])!), Float(Double(model.layer2.weight[1][3])!), Float(Double(model.layer2.weight[1][4])!),Float(Double(model.layer2.weight[1][5])!), Float(Double(model.layer2.weight[1][6])!),Float(Double(model.layer2.weight[1][7])!),Float(Double(model.layer2.weight[1][8])!),Float(Double(model.layer2.weight[1][9])!),Float(Double(model.layer2.weight[1][10])!),Float(Double(model.layer2.weight[1][11])!),Float(Double(model.layer2.weight[1][12])!), Float(Double(model.layer2.weight[1][13])!), Float(Double(model.layer2.weight[1][14])!), Float(Double(model.layer2.weight[1][15])!)]
let pesosLayer23: [Float] = [Float(Double(model.layer2.weight[2][0])!), Float(Double(model.layer2.weight[2][1])!),  Float(Double(model.layer2.weight[2][2])!), Float(Double(model.layer2.weight[2][3])!), Float(Double(model.layer2.weight[2][4])!),Float(Double(model.layer2.weight[2][5])!), Float(Double(model.layer2.weight[2][6])!),Float(Double(model.layer2.weight[2][7])!),Float(Double(model.layer2.weight[2][8])!),Float(Double(model.layer2.weight[2][9])!),Float(Double(model.layer2.weight[2][10])!),Float(Double(model.layer2.weight[2][11])!),Float(Double(model.layer2.weight[2][12])!), Float(Double(model.layer2.weight[2][13])!), Float(Double(model.layer2.weight[2][14])!), Float(Double(model.layer2.weight[2][15])!)]
let pesosLayer24 : [Float] = [Float(Double(model.layer2.weight[3][0])!), Float(Double(model.layer2.weight[3][1])!),  Float(Double(model.layer2.weight[3][2])!), Float(Double(model.layer2.weight[3][3])!), Float(Double(model.layer2.weight[3][4])!),Float(Double(model.layer2.weight[3][5])!), Float(Double(model.layer2.weight[3][6])!),Float(Double(model.layer2.weight[3][7])!),Float(Double(model.layer2.weight[3][8])!),Float(Double(model.layer2.weight[3][9])!),Float(Double(model.layer2.weight[3][10])!),Float(Double(model.layer2.weight[3][11])!),Float(Double(model.layer2.weight[3][12])!), Float(Double(model.layer2.weight[3][13])!), Float(Double(model.layer2.weight[3][14])!), Float(Double(model.layer2.weight[3][15])!)]
let pesosLayer25 : [Float] = [Float(Double(model.layer2.weight[4][0])!), Float(Double(model.layer2.weight[4][1])!),  Float(Double(model.layer2.weight[4][2])!), Float(Double(model.layer2.weight[4][3])!), Float(Double(model.layer2.weight[4][4])!),Float(Double(model.layer2.weight[4][5])!), Float(Double(model.layer2.weight[4][6])!),Float(Double(model.layer2.weight[4][7])!),Float(Double(model.layer2.weight[4][8])!),Float(Double(model.layer2.weight[4][9])!),Float(Double(model.layer2.weight[4][10])!),Float(Double(model.layer2.weight[4][11])!),Float(Double(model.layer2.weight[4][12])!), Float(Double(model.layer2.weight[4][13])!), Float(Double(model.layer2.weight[4][14])!), Float(Double(model.layer2.weight[4][15])!)]
let pesosLayer26 : [Float] = [Float(Double(model.layer2.weight[5][0])!), Float(Double(model.layer2.weight[5][1])!),  Float(Double(model.layer2.weight[5][2])!), Float(Double(model.layer2.weight[5][3])!), Float(Double(model.layer2.weight[5][4])!),Float(Double(model.layer2.weight[5][5])!), Float(Double(model.layer2.weight[5][6])!),Float(Double(model.layer2.weight[5][7])!),Float(Double(model.layer2.weight[5][8])!),Float(Double(model.layer2.weight[5][9])!),Float(Double(model.layer2.weight[5][10])!),Float(Double(model.layer2.weight[5][11])!),Float(Double(model.layer2.weight[5][12])!), Float(Double(model.layer2.weight[5][13])!), Float(Double(model.layer2.weight[5][14])!), Float(Double(model.layer2.weight[5][15])!)]
let pesosLayer27 : [Float] = [Float(Double(model.layer2.weight[6][0])!), Float(Double(model.layer2.weight[6][1])!),  Float(Double(model.layer2.weight[6][2])!), Float(Double(model.layer2.weight[6][3])!), Float(Double(model.layer2.weight[6][4])!),Float(Double(model.layer2.weight[6][5])!), Float(Double(model.layer2.weight[6][6])!),Float(Double(model.layer2.weight[6][7])!),Float(Double(model.layer2.weight[6][8])!),Float(Double(model.layer2.weight[6][9])!),Float(Double(model.layer2.weight[6][10])!),Float(Double(model.layer2.weight[6][11])!),Float(Double(model.layer2.weight[6][12])!), Float(Double(model.layer2.weight[6][13])!), Float(Double(model.layer2.weight[6][14])!), Float(Double(model.layer2.weight[6][15])!)]
let pesosLayer28 : [Float] = [Float(Double(model.layer2.weight[7][0])!), Float(Double(model.layer2.weight[7][1])!),  Float(Double(model.layer2.weight[7][2])!), Float(Double(model.layer2.weight[7][3])!), Float(Double(model.layer2.weight[7][4])!),Float(Double(model.layer2.weight[7][5])!), Float(Double(model.layer2.weight[7][6])!),Float(Double(model.layer2.weight[7][7])!),Float(Double(model.layer2.weight[7][8])!),Float(Double(model.layer2.weight[7][9])!),Float(Double(model.layer2.weight[7][10])!),Float(Double(model.layer2.weight[7][11])!),Float(Double(model.layer2.weight[7][12])!), Float(Double(model.layer2.weight[7][13])!), Float(Double(model.layer2.weight[7][14])!), Float(Double(model.layer2.weight[7][15])!)]
let pesosLayer29 : [Float] = [Float(Double(model.layer2.weight[8][0])!), Float(Double(model.layer2.weight[8][1])!),  Float(Double(model.layer2.weight[8][2])!), Float(Double(model.layer2.weight[8][3])!), Float(Double(model.layer2.weight[8][4])!),Float(Double(model.layer2.weight[8][5])!), Float(Double(model.layer2.weight[8][6])!),Float(Double(model.layer2.weight[8][7])!),Float(Double(model.layer2.weight[8][8])!),Float(Double(model.layer2.weight[8][9])!),Float(Double(model.layer2.weight[8][10])!),Float(Double(model.layer2.weight[8][11])!),Float(Double(model.layer2.weight[8][12])!), Float(Double(model.layer2.weight[8][13])!), Float(Double(model.layer2.weight[8][14])!), Float(Double(model.layer2.weight[8][15])!)]
let pesosLayer210 : [Float] = [Float(Double(model.layer2.weight[9][0])!), Float(Double(model.layer2.weight[9][1])!),  Float(Double(model.layer2.weight[9][2])!), Float(Double(model.layer2.weight[9][3])!), Float(Double(model.layer2.weight[9][4])!),Float(Double(model.layer2.weight[9][5])!), Float(Double(model.layer2.weight[9][6])!),Float(Double(model.layer2.weight[9][7])!),Float(Double(model.layer2.weight[9][8])!),Float(Double(model.layer2.weight[9][9])!),Float(Double(model.layer2.weight[9][10])!),Float(Double(model.layer2.weight[9][11])!),Float(Double(model.layer2.weight[9][12])!), Float(Double(model.layer2.weight[9][13])!), Float(Double(model.layer2.weight[9][14])!), Float(Double(model.layer2.weight[9][15])!)]
let pesosLayer211 : [Float] = [Float(Double(model.layer2.weight[10][0])!), Float(Double(model.layer2.weight[10][1])!),  Float(Double(model.layer2.weight[10][2])!), Float(Double(model.layer2.weight[10][3])!), Float(Double(model.layer2.weight[10][4])!),Float(Double(model.layer2.weight[10][5])!), Float(Double(model.layer2.weight[10][6])!),Float(Double(model.layer2.weight[10][7])!),Float(Double(model.layer2.weight[10][8])!),Float(Double(model.layer2.weight[10][9])!),Float(Double(model.layer2.weight[10][10])!),Float(Double(model.layer2.weight[10][11])!),Float(Double(model.layer2.weight[10][12])!), Float(Double(model.layer2.weight[10][13])!), Float(Double(model.layer2.weight[10][14])!), Float(Double(model.layer2.weight[10][15])!)]
let pesosLayer212 : [Float] = [Float(Double(model.layer2.weight[11][0])!), Float(Double(model.layer2.weight[11][1])!),  Float(Double(model.layer2.weight[11][2])!), Float(Double(model.layer2.weight[11][3])!), Float(Double(model.layer2.weight[11][4])!),Float(Double(model.layer2.weight[11][5])!), Float(Double(model.layer2.weight[11][6])!),Float(Double(model.layer2.weight[11][7])!),Float(Double(model.layer2.weight[11][8])!),Float(Double(model.layer2.weight[11][9])!),Float(Double(model.layer2.weight[11][10])!),Float(Double(model.layer2.weight[11][11])!),Float(Double(model.layer2.weight[11][12])!), Float(Double(model.layer2.weight[11][13])!), Float(Double(model.layer2.weight[11][14])!), Float(Double(model.layer2.weight[11][15])!)]
let pesosLayer213 : [Float] = [Float(Double(model.layer2.weight[12][0])!), Float(Double(model.layer2.weight[12][1])!),  Float(Double(model.layer2.weight[12][2])!), Float(Double(model.layer2.weight[12][3])!), Float(Double(model.layer2.weight[12][4])!),Float(Double(model.layer2.weight[12][5])!), Float(Double(model.layer2.weight[12][6])!),Float(Double(model.layer2.weight[12][7])!),Float(Double(model.layer2.weight[12][8])!),Float(Double(model.layer2.weight[12][9])!),Float(Double(model.layer2.weight[12][10])!),Float(Double(model.layer2.weight[12][11])!),Float(Double(model.layer2.weight[12][12])!), Float(Double(model.layer2.weight[12][13])!), Float(Double(model.layer2.weight[12][14])!), Float(Double(model.layer2.weight[12][15])!)]
let pesosLayer214 : [Float] = [Float(Double(model.layer2.weight[13][0])!), Float(Double(model.layer2.weight[13][1])!),  Float(Double(model.layer2.weight[13][2])!), Float(Double(model.layer2.weight[13][3])!), Float(Double(model.layer2.weight[13][4])!),Float(Double(model.layer2.weight[13][5])!), Float(Double(model.layer2.weight[13][6])!),Float(Double(model.layer2.weight[13][7])!),Float(Double(model.layer2.weight[13][8])!),Float(Double(model.layer2.weight[13][9])!),Float(Double(model.layer2.weight[13][10])!),Float(Double(model.layer2.weight[13][11])!),Float(Double(model.layer2.weight[13][12])!), Float(Double(model.layer2.weight[13][13])!), Float(Double(model.layer2.weight[13][14])!), Float(Double(model.layer2.weight[13][15])!)]
let pesosLayer215 : [Float] = [Float(Double(model.layer2.weight[14][0])!), Float(Double(model.layer2.weight[14][1])!),  Float(Double(model.layer2.weight[14][2])!), Float(Double(model.layer2.weight[14][3])!), Float(Double(model.layer2.weight[14][4])!),Float(Double(model.layer2.weight[14][5])!), Float(Double(model.layer2.weight[14][6])!),Float(Double(model.layer2.weight[14][7])!),Float(Double(model.layer2.weight[14][8])!),Float(Double(model.layer2.weight[14][9])!),Float(Double(model.layer2.weight[14][10])!),Float(Double(model.layer2.weight[14][11])!),Float(Double(model.layer2.weight[14][12])!), Float(Double(model.layer2.weight[14][13])!), Float(Double(model.layer2.weight[14][14])!), Float(Double(model.layer2.weight[14][15])!)]
let pesosLayer216 : [Float] = [Float(Double(model.layer2.weight[15][0])!), Float(Double(model.layer2.weight[15][1])!),  Float(Double(model.layer2.weight[15][2])!), Float(Double(model.layer2.weight[15][3])!), Float(Double(model.layer2.weight[15][4])!),Float(Double(model.layer2.weight[15][5])!), Float(Double(model.layer2.weight[15][6])!),Float(Double(model.layer2.weight[15][7])!),Float(Double(model.layer2.weight[15][8])!),Float(Double(model.layer2.weight[15][9])!),Float(Double(model.layer2.weight[15][10])!),Float(Double(model.layer2.weight[15][11])!),Float(Double(model.layer2.weight[15][12])!), Float(Double(model.layer2.weight[15][13])!), Float(Double(model.layer2.weight[15][14])!), Float(Double(model.layer2.weight[15][15])!)]

let pesosLayer31: [Float] = [Float(Double(model.layer3.weight[0][0])!), Float(Double(model.layer3.weight[0][1])!)]
let pesosLayer32: [Float] = [Float(Double(model.layer3.weight[1][0])!), Float(Double(model.layer3.weight[1][1])!)]
let pesosLayer33: [Float] = [Float(Double(model.layer3.weight[2][0])!), Float(Double(model.layer3.weight[2][1])!)]
let pesosLayer34: [Float] = [Float(Double(model.layer3.weight[3][0])!), Float(Double(model.layer3.weight[3][1])!)]
let pesosLayer35: [Float] = [Float(Double(model.layer3.weight[4][0])!), Float(Double(model.layer3.weight[4][1])!)]
let pesosLayer36: [Float] = [Float(Double(model.layer3.weight[5][0])!), Float(Double(model.layer3.weight[5][1])!)]
let pesosLayer37: [Float] = [Float(Double(model.layer3.weight[6][0])!), Float(Double(model.layer3.weight[6][1])!)]
let pesosLayer38: [Float] = [Float(Double(model.layer3.weight[7][0])!), Float(Double(model.layer3.weight[7][1])!)]
let pesosLayer39: [Float] = [Float(Double(model.layer3.weight[8][0])!), Float(Double(model.layer3.weight[8][1])!)]
let pesosLayer310: [Float] = [Float(Double(model.layer3.weight[9][0])!), Float(Double(model.layer3.weight[9][1])!)]
let pesosLayer311: [Float] = [Float(Double(model.layer3.weight[10][0])!), Float(Double(model.layer3.weight[10][1])!)]
let pesosLayer312: [Float] = [Float(Double(model.layer3.weight[11][0])!), Float(Double(model.layer3.weight[11][1])!)]
let pesosLayer313: [Float] = [Float(Double(model.layer3.weight[12][0])!), Float(Double(model.layer3.weight[12][1])!)]
let pesosLayer314: [Float] = [Float(Double(model.layer3.weight[13][0])!), Float(Double(model.layer3.weight[13][1])!)]
let pesosLayer315: [Float] = [Float(Double(model.layer3.weight[14][0])!), Float(Double(model.layer3.weight[14][1])!)]
let pesosLayer316: [Float] = [Float(Double(model.layer3.weight[15][0])!), Float(Double(model.layer3.weight[15][1])!)]



In [83]:
let biasLayer1 : [Float] = [Float(Double(model.layer1.bias[0])!), Float(Double(model.layer1.bias[1])!), Float(Double(model.layer1.bias[2])!), Float(Double(model.layer1.bias[3])!), Float(Double(model.layer1.bias[4])!), Float(Double(model.layer1.bias[5])!), Float(Double(model.layer1.bias[6])!), Float(Double(model.layer1.bias[7])!), Float(Double(model.layer1.bias[8])!), Float(Double(model.layer1.bias[9])!), Float(Double(model.layer1.bias[10])!)]
let biasLayer2 : [Float] = [Float(Double(model.layer2.bias[0])!), Float(Double(model.layer2.bias[1])!), Float(Double(model.layer2.bias[2])!), Float(Double(model.layer2.bias[3])!), Float(Double(model.layer2.bias[4])!), Float(Double(model.layer2.bias[5])!), Float(Double(model.layer2.bias[6])!), Float(Double(model.layer2.bias[7])!), Float(Double(model.layer2.bias[8])!), Float(Double(model.layer2.bias[9])!), Float(Double(model.layer2.bias[10])!)]
let biasLayer3 : [Float] = [Float(Double(model.layer3.bias[0])!), Float(Double(model.layer3.bias[1])!)]


In [86]:
let coreModel = CoreML_Specification_Model.with {
    $0.specificationVersion = 4
    $0.description_p = CoreML_Specification_ModelDescription.with {
        $0.input = [CoreML_Specification_FeatureDescription.with {
            $0.name = "dense_input"
            $0.type = CoreML_Specification_FeatureType.with {
                $0.multiArrayType = CoreML_Specification_ArrayFeatureType.with {
                    $0.shape = [1]
                    $0.dataType = CoreML_Specification_ArrayFeatureType.ArrayDataType.double
                }
            }
        }]
        $0.output = [CoreML_Specification_FeatureDescription.with {
            $0.name = "output"
            $0.type = CoreML_Specification_FeatureType.with {
                $0.multiArrayType = CoreML_Specification_ArrayFeatureType.with {
                    $0.shape = [1]
                    $0.dataType = CoreML_Specification_ArrayFeatureType.ArrayDataType.double
                }
            }
        }]
        $0.trainingInput = [CoreML_Specification_FeatureDescription.with {
            $0.name = "dense_input"
            $0.type = CoreML_Specification_FeatureType.with {
                $0.multiArrayType = CoreML_Specification_ArrayFeatureType.with {
                    $0.shape = [1]
                    $0.dataType = CoreML_Specification_ArrayFeatureType.ArrayDataType.double
                }
            }
        }, CoreML_Specification_FeatureDescription.with {
            $0.name = "output_true"
            $0.type = CoreML_Specification_FeatureType.with {
                $0.multiArrayType = CoreML_Specification_ArrayFeatureType.with {
                    $0.shape = [1]
                    $0.dataType = CoreML_Specification_ArrayFeatureType.ArrayDataType.double
                }
            }

        }]
        $0.metadata = CoreML_Specification_Metadata.with {
            $0.shortDescription = "Clasificador de canciones explicitas"
            $0.author = "Alejandro Hernandez Lopez"
            $0.license = "MIT"
            $0.userDefined = ["coremltoolsVersion" : "3.1"]
        }
    }
    $0.isUpdatable = true
    $0.neuralNetwork = CoreML_Specification_NeuralNetwork.with {
        $0.layers = [CoreML_Specification_NeuralNetworkLayer.with {
            $0.name = "dense_1"
            $0.input = ["dense_input"]
            $0.output = ["output"]
            $0.isUpdatable = true
            $0.innerProduct = CoreML_Specification_InnerProductLayerParams.with {
                $0.inputChannels = 11
                $0.outputChannels = 16
                $0.hasBias_p = true
                $0.weights = CoreML_Specification_WeightParams.with {
                    $0.floatValue = pesosLayer1
                    $0.isUpdatable = true
                }
                
                $0.bias = CoreML_Specification_WeightParams.with {
                    $0.floatValue = biasLayer1
                    $0.isUpdatable = true
                }
            }
        }, 
        CoreML_Specification_NeuralNetworkLayer.with {
            $0.name = "dense_2"
            $0.input = ["dense_input"]
            $0.output = ["output"]
            $0.isUpdatable = true
            $0.innerProduct = CoreML_Specification_InnerProductLayerParams.with {
                $0.inputChannels = 16
                $0.outputChannels = 16
                $0.hasBias_p = true
                $0.weights = CoreML_Specification_WeightParams.with {
                    $0.floatValue = pesosLayer21
                    $0.isUpdatable = true
                }


                $0.bias = CoreML_Specification_WeightParams.with {
                    $0.floatValue = biasLayer2
                    $0.isUpdatable = true
                }
            }
        }, 
        CoreML_Specification_NeuralNetworkLayer.with {
            $0.name = "dense_3"
            $0.input = ["dense_input"]
            $0.output = ["output"]
            $0.isUpdatable = true
            $0.innerProduct = CoreML_Specification_InnerProductLayerParams.with {
                $0.inputChannels = 16
                $0.outputChannels = 2
                $0.hasBias_p = true
                $0.weights = CoreML_Specification_WeightParams.with {
                    $0.floatValue = pesosLayer31
                    $0.isUpdatable = true
                }
                
                $0.bias = CoreML_Specification_WeightParams.with {
                    $0.floatValue = biasLayer3
                    $0.isUpdatable = true
                }
            }
        }
        ]
        $0.updateParams = CoreML_Specification_NetworkUpdateParameters.with {
            $0.lossLayers = [CoreML_Specification_LossLayer.with {
                $0.name = "lossLayer"
                $0.meanSquaredErrorLossLayer = CoreML_Specification_MeanSquaredErrorLossLayer.with {
                    $0.input = "output"
                    $0.target = "output_true"
                }
            }]
            $0.optimizer = CoreML_Specification_Optimizer.with {
                $0.sgdOptimizer = CoreML_Specification_SGDOptimizer.with {
                    $0.learningRate = CoreML_Specification_DoubleParameter.with {
                        $0.defaultValue = 0.01
                        $0.range = CoreML_Specification_DoubleRange.with {
                            $0.maxValue = 1.0
                        }
                    }
                    $0.miniBatchSize = CoreML_Specification_Int64Parameter.with {
                        $0.defaultValue = 1
                        $0.set = CoreML_Specification_Int64Set.with {
                            $0.values = [1]
                        }
                    }
                    $0.momentum = CoreML_Specification_DoubleParameter.with {
                        $0.defaultValue = 0
                        $0.range = CoreML_Specification_DoubleRange.with {
                            $0.maxValue = 1.0
                        }
                    }
                }
            }
            $0.epochs = CoreML_Specification_Int64Parameter.with {
                $0.defaultValue = 100
                $0.set = CoreML_Specification_Int64Set.with {
                    $0.values = [100]
                }
            }
            $0.shuffle = CoreML_Specification_BoolParameter.with {
                $0.defaultValue = true
            }
        }
    }
}

let binaryModelData: Data = try coreModel.serializedData()
binaryModelData.write(to: URL(fileURLWithPath: "./s4tf_model_personalization.mlmodel"))

: ignored